# Изменение формы и сводные таблицы

## Изменение формы путем поворота объектов DataFrame

![](../pic/reshaping_pivot.png)

Данные часто хранятся в так называемом "уложенном" (stacked) или "записанном" (record) формате:

Создадим [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html):

In [3]:
import numpy as np
import pandas as pd

import pandas._testing as tm

def unpivot(frame):
    N, K = frame.shape
    data = {'value': frame.to_numpy().ravel('F'),
            'variable': np.asarray(frame.columns).repeat(N),
            'date': np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])

df = unpivot(tm.makeTimeDataFrame(3))

In [4]:
df

,date,variable,value
0,2000-01-03,A,-0.114538
1,2000-01-04,A,0.041343
2,2000-01-05,A,-1.468577
3,2000-01-03,B,0.104786
4,2000-01-04,B,-1.674384
5,2000-01-05,B,0.303537
6,2000-01-03,C,0.578988
7,2000-01-04,C,-1.629925
8,2000-01-05,C,-1.076875
9,2000-01-03,D,-0.257938


Чтобы отфильтровать все строки, в которых встречается переменная `A`, мы могли бы выполнить:

In [5]:
df[df['variable'] == 'A']

,date,variable,value
0,2000-01-03,A,-0.114538
1,2000-01-04,A,0.041343
2,2000-01-05,A,-1.468577


Предположим, что мы хотим выполнить операции с [временными рядами](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html). Лучшее представление в этом случае, когда `столбцы` - уникальные переменные, а `индекс` временнЫх меток идентифицирует отдельные наблюдения. Чтобы преобразовать данные в эту форму, используем метод [`DataFrame.pivot()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html#pandas.DataFrame.pivot) (также реализован как функция верхнего уровня [`pivot()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot.html#pandas.pivot)):

In [6]:
df.pivot(index='date', columns='variable', values='value')

variable,A,B,C,D
date,,,,
2000-01-03,-0.114538,0.104786,0.578988,-0.257938
2000-01-04,0.041343,-1.674384,-1.629925,0.802044
2000-01-05,-1.468577,0.303537,-1.076875,1.027610


Если аргумент `values` не определен, а входной [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) имеет более одного столбца значений, которые не используются в качестве входных столбцов или индексов для `pivot` (поворота), то результирующий "повернутый" [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) будет иметь [`иерархические столбцы`](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced-hierarchical), самый верхний уровень которых указывает соответствующий столбец значений:

In [11]:
df['value2'] = df['value'] * 2
df

,date,variable,value,value2
0,2000-01-03,A,-0.114538,-0.229077
1,2000-01-04,A,0.041343,0.082685
2,2000-01-05,A,-1.468577,-2.937154
3,2000-01-03,B,0.104786,0.209572
4,2000-01-04,B,-1.674384,-3.348769
5,2000-01-05,B,0.303537,0.607074
6,2000-01-03,C,0.578988,1.157976
7,2000-01-04,C,-1.629925,-3.259850
8,2000-01-05,C,-1.076875,-2.153750
9,2000-01-03,D,-0.257938,-0.515875


In [9]:
pivoted = df.pivot(index='date', columns='variable')
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03 -0.114538  0.104786  0.578988 -0.257938 -0.229077  0.209572   
2000-01-04  0.041343 -1.674384 -1.629925  0.802044  0.082685 -3.348769   
2000-01-05 -1.468577  0.303537 -1.076875  1.027610 -2.937154  0.607074   

                                
variable           C         D  
date                            
2000-01-03  1.157976 -0.515875  
2000-01-04 -3.259850  1.604087  
2000-01-05 -2.153750  2.055220

Затем вы можете выбрать подмножества из повернутого (pivoted) `DataFrame`:

In [10]:
pivoted['value2']

variable,A,B,C,D
date,,,,
2000-01-03,-0.229077,0.209572,1.157976,-0.515875
2000-01-04,0.082685,-3.348769,-3.259850,1.604087
2000-01-05,-2.937154,0.607074,-2.153750,2.055220


Обратите внимание, что возвращается [представление данных](https://www.practicaldatascience.org/html/views_and_copies_in_pandas.html) в случае, когда данные одного типа.

[`pivot()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot.html#pandas.pivot) выдаст ошибку `ValueError: индекс содержит повторяющиеся записи, не может изменить форму` (`ValueError: Index contains duplicate entries, cannot reshape`), если пара индекс/столбец не уникальна.

## Изменение формы путем наложения (stacking) и разложения (unstacking)

![](../pic/reshaping_stack.png)

С методом [`pivot()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html#pandas.DataFrame.pivot) тесно связаны методы [`stack()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html#pandas.DataFrame.stack) и [`unstack()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html#pandas.DataFrame.unstack), доступные в [`Series`](https://pandas.pydata.org/pandas-docs/stable/reference/series.html) и [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). Эти методы предназначены для совместной работы с объектами `MultiIndex` (см. [Раздел об иерархической индексации](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced-hierarchical)). Вот, что в сущности делают эти методы:
- `stack`: "поворот" уровня (возможно, иерархического) меток столбцов, возвращающий [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) с индексом с новым самым внутренним уровнем меток строк.
- `unstack`: (обратная операция к `stack`) "повернуть" уровень (возможно, иерархического) индекса строки к оси столбца, создав преобразованный [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) с новым самым внутренним уровнем меток столбцов.

![](../pic/reshaping_unstack.png)

Самый ясный способ объяснить это на примере. Возьмем пример набора данных из [раздела об иерархической индексации](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html):

In [14]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
                     'foo', 'foo', 'qux', 'qux'],
                    ['one', 'two', 'one', 'two',
                    'one', 'two', 'one', 'two']]))
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

In [15]:
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           names=['first', 'second'])

In [16]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])
df

A         B
first second                    
bar   one    -1.556100  1.353979
      two    -0.998060  0.659328
baz   one     0.523203 -1.640008
      two     0.898519 -0.363958
foo   one     0.417024  0.758936
      two    -0.388472  0.955975
qux   one     0.330409  0.484554
      two    -1.773398 -0.156775

In [17]:
df2 = df[:4]
df2

A         B
first second                    
bar   one    -1.556100  1.353979
      two    -0.998060  0.659328
baz   one     0.523203 -1.640008
      two     0.898519 -0.363958

Функция stack "сжимает" уровень в столбцах [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) для получения:

- [`Series`](https://pandas.pydata.org/pandas-docs/stable/reference/series.html) - в случае простого столбца `Index`.
- [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) - в случае, если в столбцах `MultiIndex`.

Если столбцы имеют `MultiIndex`, вы можете выбрать, какой уровень складывать (to stack). Сложенный уровень становится новым самым низким уровнем в `MultiIndex` столбцов:

In [18]:
stacked = df2.stack()
stacked

first  second   
bar    one     A   -1.556100
               B    1.353979
       two     A   -0.998060
               B    0.659328
baz    one     A    0.523203
               B   -1.640008
       two     A    0.898519
               B   -0.363958
dtype: float64

Со "сложенными" (stacked) [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) или [`Series`](https://pandas.pydata.org/pandas-docs/stable/reference/series.html) (имеющими `MultiIndex` в качестве индекса) обратная операция к 
`stack` - `unstack`, которая по умолчанию разбивает стек на **последнем уровне**:

In [19]:
stacked.unstack()

A         B
first second                    
bar   one    -1.556100  1.353979
      two    -0.998060  0.659328
baz   one     0.523203 -1.640008
      two     0.898519 -0.363958

In [20]:
stacked.unstack(1)

second        one       two
first                      
bar   A -1.556100 -0.998060
      B  1.353979  0.659328
baz   A  0.523203  0.898519
      B -1.640008 -0.363958

In [21]:
stacked.unstack(0)

first          bar       baz
second                      
one    A -1.556100  0.523203
       B  1.353979 -1.640008
two    A -0.998060  0.898519
       B  0.659328 -0.363958

![](../pic/reshaping_unstack_1.png)

Если индексы имеют имена, вы можете использовать имена уровней вместо указания номеров уровней:

In [23]:
stacked.unstack('second')

second        one       two
first                      
bar   A -1.556100 -0.998060
      B  1.353979  0.659328
baz   A  0.523203  0.898519
      B -1.640008 -0.363958

![](../pic/reshaping_unstack_0.png)

Обратите внимание, что методы `stack` и `unstack` неявно сортируют участвующие уровни индекса. Следовательно, вызов `stack`, а затем `unstack` или наоборот, приведет к **отсортированной** копии исходного [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) или [`Series`](https://pandas.pydata.org/pandas-docs/stable/reference/series.html):

In [24]:
index = pd.MultiIndex.from_product([[2, 1], ['a', 'b']])
index

MultiIndex([(2, 'a'),
            (2, 'b'),
            (1, 'a'),
            (1, 'b')],
           )

In [25]:
df = pd.DataFrame(np.random.randn(4), index=index, columns=['A'])
df

A
2 a  0.133304
  b -0.965087
1 a -0.531835
  b  0.807697

In [26]:
all(df.unstack().stack() == df.sort_index())

True

Приведенный выше код вызовет ошибку `TypeError`, если вызов [`sort_index`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_index.html) будет удален.

### Несколько уровней

Вы также можете складывать (stack) или распаковывать (unstack) более одного уровня за раз, передавая список уровней, и в этом случае конечный результат будет таким, как если бы каждый уровень в списке обрабатывался индивидуально.

In [27]:
columns = pd.MultiIndex.from_tuples([
    ('A', 'cat', 'long'), ('B', 'cat', 'long'),
    ('A', 'dog', 'short'), ('B', 'dog', 'short')],
    names=['exp', 'animal', 'hair_length']
)

In [28]:
columns

MultiIndex([('A', 'cat',  'long'),
            ('B', 'cat',  'long'),
            ('A', 'dog', 'short'),
            ('B', 'dog', 'short')],
           names=['exp', 'animal', 'hair_length'])

In [29]:
df = pd.DataFrame(np.random.randn(4, 4), columns=columns)
df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,1.616750,-1.334561,0.763250,-0.891543
1,-0.333034,2.138741,-0.722436,-0.395323
2,-0.005944,2.906291,-0.981190,0.284919
3,1.144018,2.566542,0.097585,-0.487451


In [30]:
df.stack(level=['animal', 'hair_length'])

exp                          A         B
  animal hair_length                    
0 cat    long         1.616750 -1.334561
  dog    short        0.763250 -0.891543
1 cat    long        -0.333034  2.138741
  dog    short       -0.722436 -0.395323
2 cat    long        -0.005944  2.906291
  dog    short       -0.981190  0.284919
3 cat    long         1.144018  2.566542
  dog    short        0.097585 -0.487451

Список уровней может содержать либо названия уровней, либо номера уровней (но не их сочетание).

In [31]:
# df.stack(level=['animal', 'hair_length'])
# эквивалентно:
df.stack(level=[1, 2])

exp                          A         B
  animal hair_length                    
0 cat    long         1.616750 -1.334561
  dog    short        0.763250 -0.891543
1 cat    long        -0.333034  2.138741
  dog    short       -0.722436 -0.395323
2 cat    long        -0.005944  2.906291
  dog    short       -0.981190  0.284919
3 cat    long         1.144018  2.566542
  dog    short        0.097585 -0.487451

### Отсутствующие данные

Эти функции разумно обрабатывают отсутствующие данные и не ожидают, что каждая подгруппа в иерархическом индексе будет иметь одинаковый набор меток. Они также могут обрабатывать несортированный индекс (но вы можете отсортировать его, вызвав `sort_index`). 

Вот более сложный пример:

In [33]:
columns = pd.MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
                                     ('B', 'cat'), ('A', 'dog')],
                                    names=['exp', 'animal'])
columns

MultiIndex([('A', 'cat'),
            ('B', 'dog'),
            ('B', 'cat'),
            ('A', 'dog')],
           names=['exp', 'animal'])

In [34]:
index = pd.MultiIndex.from_product([('bar', 'baz', 'foo', 'qux'),
                                    ('one', 'two')],
                                   names=['first', 'second'])
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           names=['first', 'second'])

In [35]:
df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.943498 -0.899199 -1.741233  0.630064
      two     1.805755  0.415430  1.114395 -1.507714
baz   one    -0.689432  0.658158 -0.065016 -0.133841
      two    -2.112512 -0.148144 -0.470377  1.180013
foo   one     1.229366 -0.955645 -1.253364  2.295550
      two    -1.603259 -0.364166  0.463438  0.934203
qux   one     0.032677  0.490043  0.357667  1.445684
      two     0.156447  0.433836  1.404875 -0.520652

In [36]:
df2 = df.iloc[[0, 1, 2, 4, 5, 7]]
df2

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.943498 -0.899199 -1.741233  0.630064
      two     1.805755  0.415430  1.114395 -1.507714
baz   one    -0.689432  0.658158 -0.065016 -0.133841
foo   one     1.229366 -0.955645 -1.253364  2.295550
      two    -1.603259 -0.364166  0.463438  0.934203
qux   two     0.156447  0.433836  1.404875 -0.520652

Как упоминалось выше, `stack` можно вызвать с аргументом `level`, чтобы выбрать, какой уровень в столбцах складывать (to stack):

In [37]:
df2.stack('exp')

animal                 cat       dog
first second exp                    
bar   one    A    0.943498  0.630064
             B   -1.741233 -0.899199
      two    A    1.805755 -1.507714
             B    1.114395  0.415430
baz   one    A   -0.689432 -0.133841
             B   -0.065016  0.658158
foo   one    A    1.229366  2.295550
             B   -1.253364 -0.955645
      two    A   -1.603259  0.934203
             B    0.463438 -0.364166
qux   two    A    0.156447 -0.520652
             B    1.404875  0.433836

In [38]:
df2.stack('animal')

exp                         A         B
first second animal                    
bar   one    cat     0.943498 -1.741233
             dog     0.630064 -0.899199
      two    cat     1.805755  1.114395
             dog    -1.507714  0.415430
baz   one    cat    -0.689432 -0.065016
             dog    -0.133841  0.658158
foo   one    cat     1.229366 -1.253364
             dog     2.295550 -0.955645
      two    cat    -1.603259  0.463438
             dog     0.934203 -0.364166
qux   two    cat     0.156447  1.404875
             dog    -0.520652  0.433836

Распаковка (unstacking) может привести к отсутствующим значениям, если подгруппы не имеют одинаковых наборов меток. По умолчанию отсутствующие значения будут заменены значением заполнения по умолчанию для этого типа данных, `NaN` для `float`, `NaT` для различных временнЫх значений и т.д.

In [39]:
df3 = df.iloc[[0, 1, 4, 7], [1, 2]]
df3

exp                  B          
animal             dog       cat
first second                    
bar   one    -0.899199 -1.741233
      two     0.415430  1.114395
foo   one    -0.955645 -1.253364
qux   two     0.433836  1.404875

In [40]:
df3.unstack()

exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar    -0.899199  0.415430 -1.741233  1.114395
foo    -0.955645       NaN -1.253364       NaN
qux          NaN  0.433836       NaN  1.404875

В качестве альтернативы `unstack` принимает необязательный аргумент `fill_value` для указания значения в отсутствующих данных.

In [41]:
df3.unstack(fill_value=-1e9)

exp                B                                          
animal           dog                         cat              
second           one           two           one           two
first                                                         
bar    -8.991988e-01  4.154297e-01 -1.741233e+00  1.114395e+00
foo    -9.556445e-01 -1.000000e+09 -1.253364e+00 -1.000000e+09
qux    -1.000000e+09  4.338357e-01 -1.000000e+09  1.404875e+00

### С MultiIndex

Распаковка (unstacking), когда столбцы являются `MultiIndex`, также требует правильных действий:

In [42]:
df[:3].unstack(0)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
first        bar       baz       bar       baz       bar       baz       bar   
second                                                                         
one     0.943498 -0.689432 -0.899199  0.658158 -1.741233 -0.065016  0.630064   
two     1.805755       NaN  0.415430       NaN  1.114395       NaN -1.507714   

exp               
animal            
first        baz  
second            
one    -0.133841  
two          NaN

In [43]:
df2.unstack(1)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
second       one       two       one       two       one       two       one   
first                                                                          
bar     0.943498  1.805755 -0.899199  0.415430 -1.741233  1.114395  0.630064   
baz    -0.689432       NaN  0.658158       NaN -0.065016       NaN -0.133841   
foo     1.229366 -1.603259 -0.955645 -0.364166 -1.253364  0.463438  2.295550   
qux          NaN  0.156447       NaN  0.433836       NaN  1.404875       NaN   

exp               
animal            
second       two  
first             
bar    -1.507714  
baz          NaN  
foo     0.934203  
qux    -0.520652

## Изменение формы с помощью плавления (melt)

![](../pic/reshaping_melt.png)

Функция верхнего уровня [`melt()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html#pandas.melt) и соответствующий метод [`DataFrame.melt()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html#pandas.DataFrame.melt) полезны для преобразования `DataFrame` в формат, в котором один или несколько столбцов являются *переменными идентификатора* (identifier variables), в то время как все другие столбцы, считающиеся *измеряемыми переменными* (measured variables), "не повернуты" (unpivoted) к оси строк, оставив только два столбца без идентификаторов, (переменная) "variable" и (значение) "value". Имена этих столбцов можно настроить, указав параметры `var_name` и `value_name`.

Например,

In [44]:
cheese = pd.DataFrame({'first': ['John', 'Mary'],
                       'last': ['Doe', 'Bo'],
                       'height': [5.5, 6.0],
                       'weight': [130, 150]})
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [45]:
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [46]:
cheese.melt(id_vars=['first', 'last'], var_name='quantity')

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


В процессе преобразования `DataFrame` с помощью [`melt()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html#pandas.melt) индекс будет проигнорирован. Исходные значения индекса можно сохранить, установив для параметра `ignore_index` значение `False` (по умолчанию оно `True`).

*Новое в версии 1.1.0.*

In [47]:
index = pd.MultiIndex.from_tuples([('person', 'A'), ('person', 'B')])
index

MultiIndex([('person', 'A'),
            ('person', 'B')],
           )

In [48]:
cheese = pd.DataFrame({'first': ['John', 'Mary'],
                       'last': ['Doe', 'Bo'],
                       'height': [5.5, 6.0],
                       'weight': [130, 150]},
                      index=index)
cheese                      

first last  height  weight
person A  John  Doe     5.5     130
       B  Mary   Bo     6.0     150

In [49]:
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [50]:
cheese.melt(id_vars=['first', 'last'], ignore_index=False)

first last variable  value
person A  John  Doe   height    5.5
       B  Mary   Bo   height    6.0
       A  John  Doe   weight  130.0
       B  Mary   Bo   weight  150.0

Другой способ преобразования - использовать функцию [`wide_to_long()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.wide_to_long.html#pandas.wide_to_long). Она менее гибкая в отличие от [`melt()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html#pandas.melt), но более дружелюбная.

In [51]:
dft = pd.DataFrame({"A1970": {0: "a", 1: "b", 2: "c"},
                    "A1980": {0: "d", 1: "e", 2: "f"},
                    "B1970": {0: 2.5, 1: 1.2, 2: .7},
                    "B1980": {0: 3.2, 1: 1.3, 2: .1},
                    "X": dict(zip(range(3), np.random.randn(3)))
                   })
dft

,A1970,A1980,B1970,B1980,X
0,a,d,2.5,3.2,-0.441343
1,b,e,1.2,1.3,0.061965
2,c,f,0.7,0.1,1.955685


In [52]:
dft["id"] = dft.index
dft

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,-0.441343,0
1,b,e,1.2,1.3,0.061965,1
2,c,f,0.7,0.1,1.955685,2


In [53]:
pd.wide_to_long(dft, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,-0.441343,a,2.5
1,1970,0.061965,b,1.2
2,1970,1.955685,c,0.7
0,1980,-0.441343,d,3.2
1,1980,0.061965,e,1.3
2,1980,1.955685,f,0.1


## В сочетании со статистикой и GroupBy

Не должно вызывать удивления то, что объединение `pivot` / `stack` / `unstack` с `GroupBy` и базовыми статистическими функциями `Series` и `DataFrame` может привести к очень выразительным и быстрым манипуляциям с данными.

In [54]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.943498 -0.899199 -1.741233  0.630064
      two     1.805755  0.415430  1.114395 -1.507714
baz   one    -0.689432  0.658158 -0.065016 -0.133841
      two    -2.112512 -0.148144 -0.470377  1.180013
foo   one     1.229366 -0.955645 -1.253364  2.295550
      two    -1.603259 -0.364166  0.463438  0.934203
qux   one     0.032677  0.490043  0.357667  1.445684
      two     0.156447  0.433836  1.404875 -0.520652

In [55]:
df.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one    -0.398867 -0.134567
      two     1.460075 -0.546142
baz   one    -0.377224  0.262159
      two    -1.291445  0.515934
foo   one    -0.011999  0.669953
      two    -0.569910  0.285018
qux   one     0.195172  0.967864
      two     0.780661 -0.043408

In [56]:
# похожий результат, но другим путем
df.groupby(level=1, axis=1).mean()

animal             cat       dog
first second                    
bar   one    -0.398867 -0.134567
      two     1.460075 -0.546142
baz   one    -0.377224  0.262159
      two    -1.291445  0.515934
foo   one    -0.011999  0.669953
      two    -0.569910  0.285018
qux   one     0.195172  0.967864
      two     0.780661 -0.043408

In [57]:
df.stack().groupby(level=1).mean()

exp,A,B
second,,
one,0.719196,-0.426074
two,-0.208465,0.356161


In [58]:
df.mean().unstack(0)

exp,A,B
animal,,
cat,-0.029683,-0.023702
dog,0.540413,-0.046211


## Сводные таблицы (Pivot tables)

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#pivot-tables